In [1]:
import torch
import torch.nn.functional as F
import torchvision

In [2]:
device = torch.device('cuda') if torch.has_cuda else torch.device('cpu')

/tmp/ipykernel_90447/2965437891.py:1: UserWarning: 'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'
  device = torch.device('cuda') if torch.has_cuda else torch.device('cpu')


In [3]:
txn = torchvision.transforms.ToTensor()
train_dataset = torchvision.datasets.MNIST('./data', train=True, transform=txn)
test_dataset = torchvision.datasets.MNIST('./data', train=False, transform=txn)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64)

In [32]:

torchvision.transforms.Normalize() 

In [4]:
class ImgClassification(torch.nn.Module):
    def __init__(self):
        super(ImgClassification, self).__init__()
        self.conv1 = torch.nn.Conv2d(1, 1, 3) # 28 x 28 -> 26 x 26
        self.conv2 = torch.nn.Conv2d(1, 1, 3) # 26 x 26 -> 24 x 24 
        self.pool = torch.nn.MaxPool2d(2) # 12 x 12
        self.fc1 = torch.nn.Linear(144, 128)
        self.fc2 = torch.nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = self.fc2(x)
        x = F.softmax(x, dim=1)
        return x

In [68]:
model = ImgClassification()

In [64]:
model(train_dataset[0][0])

tensor([[0.1056, 0.1049, 0.0979, 0.0914, 0.1120, 0.1035, 0.0893, 0.0970, 0.0914,
         0.1069]], grad_fn=<SoftmaxBackward0>)

In [69]:
optimizer = torch.optim.Adam(model.parameters())
model.to(device)
for batch_idx, (data, target) in enumerate(train_loader):
    data, target = data.to(device), target.to(device)
    optimizer.zero_grad()
    output = model(data)
    loss = F.nll_loss(output, target)
    loss.backward()
    optimizer.step()

    if batch_idx % 10 == 0:
        print(f'loss: {loss.item()}')

loss: -0.09918611496686935
loss: -0.10149599611759186
loss: -0.10158921033143997
loss: -0.10971064120531082
loss: -0.13682223856449127
loss: -0.18449285626411438
loss: -0.35349100828170776
loss: -0.44351667165756226
loss: -0.455499529838562
loss: -0.5711482167243958
loss: -0.6180008053779602
loss: -0.623308539390564
loss: -0.6921536922454834
loss: -0.7243461012840271
loss: -0.7844183444976807
loss: -0.7786763310432434
loss: -0.7214035987854004
loss: -0.7776294946670532
loss: -0.7498915195465088
loss: -0.7745018601417542
loss: -0.7957522869110107
loss: -0.9126114249229431
loss: -0.8246760368347168
loss: -0.6849827766418457
loss: -0.8836206197738647
loss: -0.7940009832382202
loss: -0.8136528730392456
loss: -0.8724168539047241
loss: -0.8783845901489258
loss: -0.881289541721344
loss: -0.8637606501579285
loss: -0.8897256851196289
loss: -0.8742358684539795
loss: -0.876427412033081
loss: -0.9771164655685425
loss: -0.8286897540092468
loss: -0.7829371094703674
loss: -0.8229775428771973
loss: -0

In [90]:
model.eval()
test_loss = 0
correct = 0
with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        test_loss += F.nll_loss(output, target, reduction='sum').item()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()

print(f'acc: {correct / len(test_loader.dataset)}')

acc: 0.9022


In [91]:
_, (sample_data, sample_target) = next(enumerate(train_loader))
sample_data = sample_data.to(device)
sample_target = sample_target.to(device)

In [102]:
F.nll_loss(model(sample_data[0]), sample_target[0].unsqueeze(0))

tensor(-0.6246, device='cuda:0', grad_fn=<NllLossBackward0>)

In [99]:
sample_target[0].size()

torch.Size([])